In [ ]:
import os
import pickle
import numpy as np
import random
from tqdm.auto import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Flatten, Dense, Conv2DTranspose
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Layer

In [ ]:
def create_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Middle
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = UpSampling2D(size=(2, 2))(conv3)
    concat4 = concatenate([conv2, up4], axis=-1)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = concatenate([conv1, up5], axis=-1)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)
    model = Model(inputs, outputs)
    return model

In [ ]:
input_shape = (256, 256, 3)  # Adjust based on your data
num_classes = 1  # Binary classification (contrail vs. non-contrail)

# Create the U-Net model
model = create_unet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
train_contrail=np.load('/content/drive/MyDrive/train_contrail.npy')[1800:]
train_mask=np.load('/content/drive/MyDrive/train_mask.npy')[1800:]
valid_contrail=np.load('/content/drive/MyDrive/valid_contrail.npy')[600:]
valid_mask=np.load('/content/drive/MyDrive/valid_mask.npy')[600:]

In [ ]:
# train_contrail =np.asarray(train_contrail)[1200:]
# train_mask = np.asarray(train_mask)[1200:]
# valid_contrail=np.asarray(valid_contrail)[480:]
# valid_mask=np.asarray(valid_mask)[480:]

In [ ]:
print(train_contrail.shape)
print(train_mask.shape)
print(valid_contrail.shape)
print(valid_mask.shape)

(3000, 256, 256, 3)
(3000, 256, 256, 1)
(600, 256, 256, 3)
(600, 256, 256, 1)


In [ ]:
# Define the number of epochs and batch size
epochs = 10
batch_size = 8
# Train the model
history = model.fit(train_contrail, train_mask,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(valid_contrail, valid_mask))

Epoch 1/10
250/250 [==============================] - 92s 284ms/step - loss: 0.0421 - accuracy: 0.9911 - val_loss: 0.0278 - val_accuracy: 0.9955
Epoch 2/10
250/250 [==============================] - 70s 280ms/step - loss: 0.0303 - accuracy: 0.9951 - val_loss: 0.0382 - val_accuracy: 0.9955
Epoch 3/10
250/250 [==============================] - 70s 281ms/step - loss: 0.0301 - accuracy: 0.9951 - val_loss: 0.0260 - val_accuracy: 0.9955
Epoch 4/10
250/250 [==============================] - 71s 282ms/step - loss: 0.0272 - accuracy: 0.9951 - val_loss: 0.0240 - val_accuracy: 0.9955
Epoch 5/10
250/250 [==============================] - 70s 280ms/step - loss: 0.0265 - accuracy: 0.9951 - val_loss: 0.0246 - val_accuracy: 0.9955
Epoch 6/10
250/250 [==============================] - 70s 280ms/step - loss: 0.0246 - accuracy: 0.9951 - val_loss: 0.0229 - val_accuracy: 0.9955
Epoch 7/10
250/250 [==============================] - 70s 280ms/step - loss: 0.0241 - accuracy: 0.9951 - val_loss: 0.0221 - val_ac

In [ ]:
model.save("u-net.h5")

In [ ]:
u_net_predict =model.predict(valid_contrail[:20])

In [ ]:
class RoiPooling(Layer):
    def __init__(self, pool_size, **kwargs):
        super(RoiPooling, self).__init__(**kwargs)
        self.pool_size = pool_size

    def call(self, inputs):
        feature_map, proposals = inputs
        pooled_regions = []

        for proposal in proposals:
            y1, x1, y2, x2 = tf.unstack(proposal)
            y1 = tf.cast(y1, tf.int32)
            x1 = tf.cast(x1, tf.int32)
            y2 = tf.cast(y2, tf.int32)
            x2 = tf.cast(x2, tf.int32)

            roi = feature_map[y1:y2, x1:x2, :]
            pooled = tf.image.resize(roi, self.pool_size)
            pooled_regions.append(pooled)

        return tf.stack(pooled_regions)

In [ ]:
class GenerateProposals(Layer):
    def call(self, inputs):
        rpn_scores, rpn_bboxes = inputs
        batch_size = tf.shape(rpn_scores)[0]
        num_anchors = tf.shape(rpn_scores)[1]

        # Reshape rpn_scores and rpn_bboxes
        rpn_scores_reshaped = tf.reshape(rpn_scores, (batch_size, -1, 2))
        rpn_bboxes_reshaped = tf.reshape(rpn_bboxes, (batch_size, -1, 4))

        # Apply non-max suppression for each image in the batch
        proposals = []
        for i in range(batch_size):
            proposals_single_image = tf.image.non_max_suppression(
                rpn_bboxes_reshaped[i], rpn_scores_reshaped[i], max_output_size=num_anchors, iou_threshold=0.7
            )
            proposals.append(proposals_single_image)

        proposals = tf.stack(proposals)
        return proposals

In [ ]:
num_anchors = 9
def build_mask_rcnn_model(num_classes):
    inputs = Input(shape=(256, 256, 3))

    # Backbone
    backbone = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    x = backbone.output

    x = Conv2D(512, 3, activation='relu', padding='same')(x)
    x = Conv2D(512, 3, activation='relu', padding='same')(x)

    rpn_scores = Conv2D(num_anchors * 2, 1)(x)
    rpn_bboxes = Conv2D(num_anchors * 4, 1)(x)

    # Generate ROI proposals
    proposals = GenerateProposals()(inputs=[rpn_scores, rpn_bboxes])

  # ROI feature extraction
    x = RoiPooling()(backbone.output, proposals)
    x = Conv2D(256, 3, activation='relu')(x)

    # Mask head
    x_mask = Conv2D(256, 3, activation='relu')(x)
    x_mask = Conv2DTranspose(128, 2, strides=2, activation='relu')(x_mask)
    masks = Conv2D(1, 1, activation='sigmoid', name='masks')(x_mask)

    # Bounding box regression head
    x_boxes = Conv2D(256, 3, activation='relu')(x)
    boxes = Dense(4, name='boxes')(x_boxes)

    # Classification head
    x_cls = Flatten()(x)
    classes = Dense(num_classes, activation='softmax', name='classes')(x_cls)

    # Create model
    model = Model(inputs, [boxes, classes, masks], name='mask_rcnn')

    # Losses
    lambda_rpn = 1.0
    lambda_boxes = 1.0
    lambda_masks = 1.0

    losses = {
      'boxes': lambda_boxes * bbox_loss,
      'classes': lambda_rpn * objectness_loss + lambda_boxes * class_loss,
      'masks': lambda_masks * mask_loss
    }

    # Compile
    model.compile(optimizer='adam', loss=losses)

    return model

In [ ]:
model = build_mask_rcnn_model(1)

ValueError: ignored

In [ ]:
def build_mask_rcnn_model():
    inputs = Input(shape=(256, 256, 3))
    x = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(2)(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(2)(x)
    masks = Conv2D(1, 1, activation='sigmoid')(x)
    model = Model(inputs, masks)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Build and compile the model
model = build_mask_rcnn_model()

# Train the model
model.fit(train_contrail, train_mask, validation_data=(valid_contrail, valid_mask), epochs=10, batch_size=16)

Epoch 1/10
  5/188 [..............................] - ETA: 16:44 - loss: 0.3671

KeyboardInterrupt: ignored